In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
import geopandas as gpd
import contextily as ctx
import matplotlib.patheffects as pe
import matplotlib.ticker as mticker
import numpy as np

In [ ]:
city_data = {
    "Rank": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "CITY_NAME": [
        "Stuttgart", "Karlsruhe", "Mannheim", "Freiburg im Breisgau",
        "Heidelberg", "Ulm", "Heilbronn", "Pforzheim",
        "Reutlingen",
        "Esslingen am Neckar",
        "Ravensburg", "Konstanz"
    ],
    "Population (approx.)": [
        635000, 310000, 310000, 230000,
        160000, 125000, 125000, 125000,
        115000, 95000, None, None,
    ],
    "lat": [
        48.7758, 49.0069, 49.4875, 47.9990,
        49.3988, 48.4011, 49.1427, 48.8922,
        48.4914, 48.7405, 47.78280822171495, 47.67505184676883,
    ],
    "lon": [
        9.1829, 8.4037, 8.4660, 7.8421,
        8.6724, 9.9876, 9.2109, 8.6946,
        9.2116, 9.3114, 9.611101464378356, 9.172817383708908,
    ]
}

cities = pd.DataFrame(city_data)
cities = cities[~cities["CITY_NAME"].isin(["Pforzheim", "Reutlingen", "Esslingen am Neckar"])]

cities = gpd.GeoDataFrame(cities, crs=4326, geometry=gpd.points_from_xy(x=cities.lon, y=cities.lat))
cities = cities.to_crs(epsg=4326)

In [ ]:
states = gpd.read_file(f"data/geodata/states.gpkg")
states = states[states["NAME_1"] == "Baden-Württemberg"]
states = states.to_crs(epsg=4326)

In [ ]:
features = pd.read_csv("results/features.csv")
features.dropna(inplace=True)

In [ ]:


import sklearn


def plot_on_map(labels, savepath, filename):

    node_features = pd.read_csv("results/features.csv")
    node_features["label"] = labels

    data = pd.read_csv("./data/data-sample.csv")
    data["geometry"] = gpd.points_from_xy(x=data.lons, y=data.lats)

    gdf = gpd.GeoDataFrame(data, geometry="geometry")
    gdf = gdf.set_crs(epsg=4326)

    merged_df = gdf.merge(node_features, on='id', how='inner')

    merged_gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')
    merged_gdf = merged_gdf.set_crs(epsg=4326)

    merged_gdf["x"] = merged_gdf.geometry.x
    merged_gdf["y"] = merged_gdf.geometry.y

    fig, ax = plt.subplots(figsize=(24, 24))

    basemap = ctx.providers.CartoDB.PositronNoLabels

    sns.scatterplot(ax=ax, data=merged_gdf, x="x", y="y", hue=merged_gdf["label"], palette="tab10")
    ctx.add_basemap(ax=ax, crs=merged_gdf.crs.to_string(), source=basemap, alpha=0.5)
    states.plot(ax=ax, color="none", edgecolor="grey", linewidth=0.3)
    plt.xlabel("longitude")
    plt.ylabel("latitude")

    plt.savefig(savepath / filename)
    plt.close()

    for l in np.unique(labels):
        fig, ax = plt.subplots(figsize=(24, 24), dpi=100)

        sns.scatterplot(ax=ax, data=merged_gdf[merged_gdf["label"] == l], x="x", y="y")
        states.plot(ax=ax, color="none", edgecolor="black", linewidth=0.5)

        ax.plot(cities.geometry.x, cities.geometry.y, linestyle="None", marker="o", color="#262626", ms=5)
        for _, row in cities.iterrows():
            ax.annotate(
                row["CITY_NAME"],
                (row.geometry.x, row.geometry.y),
                color="black",
                fontsize=40,
                alpha=0.8,
                path_effects=[
                    pe.withStroke(linewidth=4, foreground="#ebebeb")
                ]
            )

        basemap = ctx.providers.CartoDB.PositronNoLabels
        ctx.add_basemap(ax=ax, crs=merged_gdf.crs.to_string(), source=basemap, alpha=0.5)
        plt.xticks([])
        plt.yticks([])
        plt.tight_layout()
        plt.savefig(savepath / f"geo_label_{l}.png")
        plt.close()


def print_optics_result(folder_path, min_samples, metric):
    sns.set(font_scale=1)
    clustering_df = pd.read_csv(folder_path / "optics_result_data.csv")
    reachability = clustering_df["reachability"].values
    ordering = clustering_df["ordering"].values
    core_distances = clustering_df["core_distances"].values

    reachability = reachability[ordering]
    fig, ax = plt.subplots(figsize=(12, 6), dpi=100)
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{int(x):,}'))
    ax.plot(reachability)
    if metric == "minkowski":
        ax.set_ylim(0, 4)
    eps_select = 1.1
    ax.hlines(eps_select, 0, len(reachability), color='red')

    labels_select = sklearn.cluster.cluster_optics_dbscan(
        reachability=clustering_df["reachability"].values,
        core_distances=core_distances,
        ordering=ordering,
        eps=eps_select,
    )

    new_node_features = pd.read_csv("results/features.csv")

    new_node_features["label"] = labels_select

    merged = new_node_features
    summary_list = list()
    for c in ['3d_printing_intensity', 'ai_intensity', 'innoprob',
              'sustainability_intensity', 'employees', 'age_years', 'outdegree', 'indegree']:

        for label, x in merged.groupby("label"):
            stats = x[c].describe()
            summary_list.append({
                "Feature": c,
                "Cluster": label,
                "Count": stats["count"],
                "Mean": stats["mean"],
                "Std": stats["std"],
                "Min": stats["min"],
                "25%": stats["25%"],
                "Median (50%)": stats["50%"],
                "75%": stats["75%"],
                "Max": stats["max"]
            })
    summary_df = pd.DataFrame(summary_list)
    summary_df.to_csv("summary.csv")

    #plt.title(f"min_samples:{min_samples}, metric:{metric}",fontsize=20)

    cluster_boundaries = []

    l_order = labels_select[ordering]
    for i in range(1, len(l_order)):
        if l_order[i] != l_order[i - 1]:
            cluster_boundaries.append(i)

    for boundary in cluster_boundaries:
        plt.axvline(x=boundary, color='red', linestyle='--')

        if l_order[boundary] != -1:
            plt.text(boundary + 1, reachability[boundary] + 0.05, f'Cluster {l_order[boundary]}', fontsize=16,
                     color='black')

    save_path = Path("results/optics_plots")
    save_path.mkdir(exist_ok=True, parents=True)
    plt.xlabel("datapoints", fontsize=16)
    plt.ylabel("reachability", fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.tight_layout()
    plt.savefig(save_path / f"optics_result_min_samples_{min_samples}_metric_{metric}.png")
    
    plot_on_map(labels_select, save_path, f"geo_optics_result_min_samples_{min_samples}_metric_{metric}.png")


optics_path = Path("results/optics")

for folder in optics_path.iterdir():
    split = folder.stem.split("-")
    metric = split[2]
    min_samples = int(split[1])
    folder_path = optics_path / f"run-{min_samples}-{metric}"
    print_optics_result(folder_path, min_samples, metric)

In [ ]:
def print_tsne_results(folder_path, perpexity, max_iter, metric):
    emb_df = pd.read_csv(folder_path / "embeddings.csv")
    fig, ax = plt.subplots(figsize=(24, 24), dpi=100)
    sns.scatterplot(x="x", y="y", data=emb_df, ax=ax)
    plt.title(f"perplexity:{perpexity}, max_iter:{max_iter}, metric:{metric}")
    save_path = Path("data/results/tsne_emb_plots")
    save_path.mkdir(exist_ok=True, parents=True)
    plt.savefig(save_path / f"tsne_emb_perp_{perpexity}_max_iter_{max_iter}_metric_{metric}.png")


tsne_path = Path("results/tsne")

for folder in tsne_path.iterdir():
    split = folder.stem.split("-")
    iter = split[2]
    perplexity = split[1]
    metric = split[3]
    folder_path = tsne_path / f"tsne-{perplexity}-{iter}-{metric}"
    print_tsne_results(folder_path, perplexity, iter, metric)


In [ ]:
import numpy as np
from collections import Counter
import geopandas as gpd
import contextily as ctx

tsne_emb_to_cluster = "tsne-140-2000-cosine"
p = Path(f"results/tsne/{tsne_emb_to_cluster}")
df = pd.read_csv(p / "embeddings.csv")


node_features = pd.read_csv("results/features.csv")


data = pd.read_csv("./data/data-sample.csv")
data["geometry"] = gpd.points_from_xy(x=data.lons, y=data.lats)

gdf = gpd.GeoDataFrame(data, geometry="geometry")
gdf = gdf.set_crs(epsg=4326)

merged_df = gdf.merge(node_features, on='id', how='inner')

merged_gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')
merged_gdf = merged_gdf.set_crs(epsg=4326)

merged_gdf["x"] = merged_gdf.geometry.x
merged_gdf["y"] = merged_gdf.geometry.y

node_features["emb_x"] = df["x"]
node_features["emb_y"] = df["y"]

merged_df = gdf.merge(node_features, on='id', how='inner')

merged_gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')
merged_gdf = merged_gdf.set_crs(epsg=4326)

merged_gdf["x"] = merged_gdf.geometry.x
merged_gdf["y"] = merged_gdf.geometry.y
merged_gdf = merged_gdf.dropna(subset=['emb_x', 'emb_y'])

states = gpd.read_file(f"data/geodata/states.gpkg")
states = states[states["NAME_1"] == "Baden-Württemberg"]
states = states.to_crs(epsg=4326)

tsne_save_path = Path("results/tsne_clustering_results")
tsne_save_path.mkdir(exist_ok=True, parents=True)

for eps in [3, 3.5, 4]:
    for min_sample in [75, 100, 125]:

        clustering = DBSCAN(eps=eps, min_samples=min_sample).fit(merged_gdf[['emb_x', 'emb_y']].values)
        labels = clustering.labels_
        label_counts = Counter(labels)
        threshold = 300
        updated_labels = np.array([label if label_counts[label] >= threshold else -1 for label in labels])
        merged_gdf["label"] = updated_labels

        fig, ax = plt.subplots(figsize=(24, 24), dpi=100)
        sns.set(font_scale=1)
        sns.scatterplot(data=merged_gdf, x="emb_x", y="emb_y", hue=updated_labels,
                        palette=sns.color_palette("husl", 100))

        centroids = merged_gdf.groupby("label")[["emb_x", "emb_y"]].mean()
        for cluster, (x, y) in centroids.iterrows():
            plt.text(x, y, str(cluster), fontsize=24, fontweight='bold',
                     bbox=dict(facecolor='white', alpha=0.6, edgecolor='black'))

        plt.legend(fontsize=20)
        plt.xticks(fontsize=40)
        plt.yticks(fontsize=40)
        plt.xlabel("emb_x", fontsize=40)
        plt.ylabel("emb_y", fontsize=40)
        plt.tight_layout()
        plt.savefig(tsne_save_path / f"{tsne_emb_to_cluster}_eps_{eps}_min_sample_{min_sample}.png")
        plt.close()

        fig, ax = plt.subplots(figsize=(24, 24), dpi=100)

        basemap = ctx.providers.CartoDB.PositronNoLabels

        sns.scatterplot(ax=ax, data=merged_gdf, x="x", y="y", hue=updated_labels,
                        palette=sns.color_palette("husl", 100))
        states.plot(ax=ax, color="none", edgecolor="black", linewidth=0.5)
        for _, row in cities.iterrows():
            ax.annotate(
                row["CITY_NAME"],
                (row.geometry.x, row.geometry.y),
                color="black",
                fontsize=40,
                alpha=0.8,
                path_effects=[
                    pe.withStroke(linewidth=4, foreground="#ebebeb")
                ]
            )

        ctx.add_basemap(ax=ax, crs=merged_gdf.crs.to_string(), source=basemap, alpha=0.5)
        states.plot(ax=ax, color="none", edgecolor="grey", linewidth=0.3)

        plt.xticks([])
        plt.yticks([])
        plt.legend(fontsize=20)

        plt.tight_layout()
        plt.savefig(tsne_save_path / f"geo_{tsne_emb_to_cluster}_eps_{eps}_min_sample_{min_sample}.png")
        plt.close()

        #tsne print per cluster on map
        for l in np.unique(updated_labels):
            fig, ax = plt.subplots(figsize=(24, 24))

            basemap = ctx.providers.CartoDB.PositronNoLabels

            sns.scatterplot(ax=ax, data=merged_gdf[merged_gdf["label"] == l], x="x", y="y")
            ctx.add_basemap(ax=ax, crs=merged_gdf.crs.to_string(), source=basemap, alpha=0.5)
            states.plot(ax=ax, color="none", edgecolor="grey", linewidth=0.3)
            plt.xlabel("longitude", fontsize=40)
            plt.ylabel("latitude", fontsize=40)
            plt.xticks(fontsize=40)
            plt.yticks(fontsize=40)
            plt.tight_layout()
            plt.savefig(tsne_save_path / f"geo_label_{l}_{tsne_emb_to_cluster}_eps_{eps}_min_sample_{min_sample}.png")
            plt.close()